# Importing library

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import hazm
from collections import defaultdict




## Load data

* In this block we load csv file and stop words

In [2]:
data = pd.read_csv('./culture.csv')
with open('Verb.txt') as f:
    verb = f.readlines()
verb = [unicode(i.strip(),'utf-8') for i in verb]

## Delete stop words and punctuation marks

In [3]:
def delete_stop_words(word):    
    if (word in verb):
        return False
    if (len(word)<3):
        return False
    else:
        n = re.sub(u'[\u060c|.|:|;|!|@|#|$|%|^|&|*|\xab|\xbb|/]','',word)
    return n

## Cleaning the thex

- This function tokenize the text's word and use delete_stop_words for each word to delete or keep the word in a list 
- Then return a list contain word and postion of each one

In [4]:
def clean_text(text):
    normalizer = hazm.Normalizer()
    text = normalizer.affix_spacing(text)
    sentence = nltk.word_tokenize(unicode(text,'utf-8'))
    clean_words = [] 
    for i in range(len(sentence)):
        res = delete_stop_words(sentence[i])
        if res:
            clean_words.append({'Position':i,'Word':res})
    return clean_words

## Take a sample
- Take a sample from all data and clean it . "words" variable contain the all words we need

In [93]:
text = data['mText'][500]
all_words = nltk.word_tokenize(unicode(text,'utf-8'))
words = clean_text(text)

In [94]:
print(text)

به گزارش خبرنگار تجسمی خبرگزاری فارس، بیش از ۵۳۰ گالری در کشورمان مجوز فعالیت دارند که از این تعداد، حدود ۲۶۰ گالری در تهران و ۲۷۰ گالری در شهرستان‌ها هستند (+). هرچند درباره گالری‌های فعال که در معرفی هنرمندان و گردش آزاد اقتصاد هنر نقش دارند،  آمار دقیقی وجود ندارد، اما با این حال تمایز بارزی میان گالری‌های پایتخت با گالری‌های استانی وجود دارد و در این باره هنرمندان به یک میزان از عدالت برخوردار نیستند. حتی در گالری‌های تهران هم هر هنرمندی نمی‌تواند نمایشگاه برگزار کند و عنصر تعیین‌کننده، گالری‌دارها هستند. برای واکاوی این مسأله و ارائه راه‌حل‌های احتمالی، در سلسله گفت‌وگوهایی با افراد صاحب‌نظر و دخیل در این ماجرا گفت‌وگو می‌کنیم.
* نظام توزیع در چرخه اقتصاد هنر معیوب است
مجتبی آقایی، دبیر ۵ دوره جشنواره هنرهای تجسمی فجر در این باره گفت: مسأله اصلی، نقصان در مدیریت کلان کشور است. به بیان دیگر، توزیع و پراکندگی مراکز فرهنگی هنری که به آثار هنری بها دهند، در کشور به خوبی انجام نشده و اکثر گالری‌های استانی در اختیار مراکز دولتی است که یا بلااستفاده می‌مانند یا در برپایی نمایشگاه، موضوع 

In [95]:
lemmet = hazm.Lemmatizer()
for i in words:
    i['Word'] = lemmet.lemmatize(i['Word'])

In [96]:
position_dict = defaultdict(list)
for i in words:
    position_dict[i['Word']].append(i['Position'])
    
for i in position_dict:
    position_dict[i] = sorted(position_dict[i])

In [97]:
life_time = defaultdict(int)
for i in position_dict:
    n = position_dict[i][-1]-position_dict[i][0] -1 
    if n == -1 :
        n = 0
    life_time[i] = float(n)/len(all_words)

In [98]:
words_dict = defaultdict()
for i in position_dict:
    n = defaultdict()
#     n['Word'] = i
    n['Position'] = position_dict[i]
    n['life_time'] = life_time[i]
    n['tf'] = len(position_dict[i])
    words_dict[i] = n

In [99]:
mean = float(sum([words_dict[i]['tf'] for i in words_dict]))/len(words_dict)
for word in words_dict:

    words_dict[word]['std'] = words_dict[word]['tf'] - mean

In [100]:
alpha = 0.55
for word in words_dict:
    words_dict[word]['Weight'] = (((1-alpha)*words_dict[word]['life_time']) + alpha*(words_dict[word]['tf']+words_dict[word]['std'])) - words_dict[word]['std']

    

In [101]:
sorted_list = sorted(words_dict,key = lambda x : words_dict[x]['Weight'],reverse=True)

In [102]:
w = nltk.word_tokenize(unicode(text,'utf-8'))

In [103]:
for i in sorted_list:
    print(i)
    print(words_dict[i])

هنر
defaultdict(None, {'tf': 22, 'Position': [46, 125, 134, 159, 163, 193, 205, 219, 238, 271, 280, 300, 427, 434, 439, 450, 462, 466, 500, 510, 618, 635], 'std': 20.257731958762886, 'life_time': 0.8977099236641222, 'Weight': 3.387990084205562})
هنرمند
defaultdict(None, {'tf': 9, 'Position': [41, 71, 86, 229, 324, 345, 397, 566, 626], 'std': 7.257731958762887, 'life_time': 0.8916030534351145, 'Weight': 2.085241992602504})
جشنواره
defaultdict(None, {'tf': 7, 'Position': [133, 453, 509, 540, 553, 586, 617], 'std': 5.257731958762887, 'life_time': 0.7374045801526717, 'Weight': 1.8158526796254035})
تجسم
defaultdict(None, {'tf': 6, 'Position': [3, 135, 454, 467, 619, 627], 'std': 4.257731958762887, 'life_time': 0.9511450381679389, 'Weight': 1.812035885732274})
کشور
defaultdict(None, {'tf': 6, 'Position': [148, 167, 207, 221, 228, 636], 'std': 4.257731958762887, 'life_time': 0.7435114503816794, 'Weight': 1.7186007712284574})
آثار
defaultdict(None, {'tf': 7, 'Position': [162, 270, 279, 295, 34

In [104]:
related_word = defaultdict(list)
for word in sorted_list[:30]:
    l_pos = 0 
    for pos in words_dict[word]['Position']:
        if pos-5 < l_pos+5:
            pos = l_pos + 10
        l_pos = pos
        related_word[word].append(w[pos-5:pos+5])

In [105]:
for i in related_word:
    t = []
    for j in related_word[i]:
        t+= j
    related_word[i] = [lemmet.lemmatize(re.sub(u'[\u060c|.|:|;|!|@|#|$|%|^|&|*|\xab|\xbb|/]','',word)).strip() for word in t ]

In [106]:
n = defaultdict()
for i in related_word:
    m = pd.Series(related_word[i]).value_counts()
    n[i] = dict(m)
    


In [107]:
probability = defaultdict(dict)
for i in n:
    for j in n[i]:
        if (j in sorted_list[:30]):
            try:
                if(i == j) or (re.search(r'#',j)):
                    continue
                prob = (float(n[i][j])/words_dict[j]['tf'])
                probability[i][j] = prob

            except:
                probability[i][j] = 0


In [108]:
for i in probability:
    sort = sorted(probability[i].items(),key=lambda x : x[1],reverse=True)
    probability[i] = sort


In [110]:
sent_dict = defaultdict()
for i in hazm.SentenceTokenizer().tokenize(unicode(text,'utf-8')):
    sentWeight = 0
    if len(nltk.word_tokenize(i)) > 25:
        continue
    for j in nltk.word_tokenize(i):
        try:
            sentWeight += words_dict[lemmet.lemmatize(j)]['Weight']
        except:
            sentWeight += 0 
    
    sent_dict[i] = sentWeight
    

In [111]:
for i in sorted(sent_dict,key=lambda x:sent_dict[x],reverse=True):
    print(i)
    print(sent_dict[i])
    break

وی با اشاره به چرخه ناقص هنر در کشور گفت: «تولید، توزیع و مصرف» سه ضلع چرخه هنر در کشور هستند.
20.0708428425
